In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
     .master("local[*]") \
     .appName("test") \
     .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/03/05 22:03:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df1 = spark.range(2, 10000000, 2)

In [3]:
df1

DataFrame[id: bigint]

In [4]:
df1.explain()

== Physical Plan ==
*(1) Range (2, 10000000, step=2, splits=2)




In [5]:
step1 = df1.repartition(5)

In [6]:
step1

DataFrame[id: bigint]

In [7]:
step1.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Exchange RoundRobinPartitioning(5), REPARTITION_BY_NUM, [plan_id=10]
   +- Range (2, 10000000, step=2, splits=2)




In [18]:
step1.show(10)

+-------+
|     id|
+-------+
|3826942|
|1976646|
|1057520|
|3209254|
| 484808|
|2383826|
|4697224|
| 652718|
|3298424|
|4469482|
+-------+
only showing top 10 rows



In [ ]:
df2 = spark.range(2, 10000000, 4)

In [8]:
step12= df2.repartition(6)

In [9]:
step2

DataFrame[id: bigint]

In [10]:
step2.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Project [(id#0L * 5) AS id#8L]
   +- Exchange RoundRobinPartitioning(5), REPARTITION_BY_NUM, [plan_id=21]
      +- Range (2, 10000000, step=2, splits=2)




In [11]:
step3 = step2.join(step12, ["id"])

In [13]:
step3.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Project [id#8L]
   +- SortMergeJoin [id#8L], [id#2L], Inner
      :- Sort [id#8L ASC NULLS FIRST], false, 0
      :  +- Exchange hashpartitioning(id#8L, 200), ENSURE_REQUIREMENTS, [plan_id=53]
      :     +- Project [(id#0L * 5) AS id#8L]
      :        +- Exchange RoundRobinPartitioning(5), REPARTITION_BY_NUM, [plan_id=45]
      :           +- Range (2, 10000000, step=2, splits=2)
      +- Sort [id#2L ASC NULLS FIRST], false, 0
         +- Exchange hashpartitioning(id#2L, 200), ENSURE_REQUIREMENTS, [plan_id=54]
            +- Exchange RoundRobinPartitioning(6), REPARTITION_BY_NUM, [plan_id=48]
               +- Range (2, 10000000, step=4, splits=2)




In [14]:
step4 = step3.selectExpr("sum(id)")

In [15]:
step4.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[], functions=[sum(id#8L)])
   +- Exchange SinglePartition, ENSURE_REQUIREMENTS, [plan_id=103]
      +- HashAggregate(keys=[], functions=[partial_sum(id#8L)])
         +- Project [id#8L]
            +- SortMergeJoin [id#8L], [id#2L], Inner
               :- Sort [id#8L ASC NULLS FIRST], false, 0
               :  +- Exchange hashpartitioning(id#8L, 200), ENSURE_REQUIREMENTS, [plan_id=95]
               :     +- Project [(id#0L * 5) AS id#8L]
               :        +- Exchange RoundRobinPartitioning(5), REPARTITION_BY_NUM, [plan_id=85]
               :           +- Range (2, 10000000, step=2, splits=2)
               +- Sort [id#2L ASC NULLS FIRST], false, 0
                  +- Exchange hashpartitioning(id#2L, 200), ENSURE_REQUIREMENTS, [plan_id=96]
                     +- Exchange RoundRobinPartitioning(6), REPARTITION_BY_NUM, [plan_id=88]
                        +- Range (2, 10000000, step=4, splits=2)




In [16]:
step4.collect()

[Row(sum(id)=2500000000000)]

In [19]:
df = spark.range(2, 10000000, 2)

In [20]:
df

DataFrame[id: bigint]

In [21]:
df.explain()

== Physical Plan ==
*(1) Range (2, 10000000, step=2, splits=2)




In [22]:
transformation = df.repartition(6)

In [23]:
transformation.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Exchange RoundRobinPartitioning(6), REPARTITION_BY_NUM, [plan_id=428]
   +- Range (2, 10000000, step=2, splits=2)




In [24]:
transformation.show(15)

+-------+
|     id|
+-------+
|1783094|
|4010386|
|3673730|
| 484808|
|3830824|
|3242912|
|2477254|
|3699514|
| 429662|
|1987292|
| 916224|
| 487040|
|1470878|
|2060572|
|2148172|
+-------+
only showing top 15 rows

